<a href="https://colab.research.google.com/github/RahulSundar/CS6910-DeepLearningFundamentals/blob/main/Assignment2/Assignment2_ObjectDetection_unbalanced_sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os, sys
sys.path.append(
"/content/drive/MyDrive/CS6910/Assignment2"
)


In [4]:
!pip install wandb

In [5]:
import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras

# keras pre-trained models
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2 as IRV2
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception


from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten, Conv2D, BatchNormalization, MaxPooling2D, Activation 
from tensorflow.keras.models import Sequential,  Model

import wandb


class ObjectDetection():

    def __init__(self, IMG_SIZE, modelConfigDict, using_pretrained_model = False, base_model = "IRV2" ):
        
        self.num_hidden_cnn_layers= modelConfigDict["num_hidden_cnn_layers"]
        self.activation = modelConfigDict["activation"]
        self.batch_normalization = modelConfigDict["batch_normalization"]
        self.filter_distribution = modelConfigDict["filter_distribution"]
        self.filter_size = modelConfigDict["filter_size"]
        self.number_of_filters_base  = modelConfigDict["number_of_filters_base"]
        self.initializer = modelConfigDict["initializer"]
        self.dropout_fraction = modelConfigDict["dropout_fraction"]
        self.pool_size = modelConfigDict["pool_size"]
        self.padding = modelConfigDict["padding"]
        self.dense_neurons = modelConfigDict["dense_neurons"]
        self.num_classes = modelConfigDict["num_classes"]
        self.optimizer = modelConfigDict["optimizer"]

        BASE_MODELS = {
                          "IRV2" : IRV2,
                          "IV3" : InceptionV3,
                          "RN50" : ResNet50,
                          "XCPTN" : Xception
                      }      
        
        if using_pretrained_model == True:
            self.base_model = base_model
            if self.base_model == "RN50":
                self.IMG_HEIGHT = 224
                self.IMG_WIDTH = 224
            else:
                self.IMG_HEIGHT = IMG_SIZE[0]
                self.IMG_WIDTH = IMG_SIZE[1]        

        self.IMG_HEIGHT = IMG_SIZE[0]
        self.IMG_WIDTH = IMG_SIZE[1]        
         
        self.input_shape = (self.IMG_HEIGHT, self.IMG_WIDTH, 3)


    def build_cnndropmodel(self):
        keras.backend.clear_session()
        model = Sequential()
        
        #First CNN layer connecting to input layer
        model.add(Conv2D(self.number_of_filters_base, self.filter_size, kernel_regularizer='l2',padding = self.padding, input_shape = (self.IMG_HEIGHT, self.IMG_WIDTH, 3)))
        model.add(Activation(self.activation))
        
        #batch_normalisation
        if self.batch_normalization: model.add(BatchNormalization())
        #max pooling
        model.add(MaxPooling2D(pool_size=self.pool_size))  
        if self.dropout_fraction != None:
            model.add(tf.keras.layers.Dropout(self.dropout_fraction))
        for i in range(self.num_hidden_cnn_layers-1):
            #i+2th Convolutional Layer
        
            ## Standard filter distribution - same number of filters in all Convolutional layers
            if self.filter_distribution == "standard":
                model.add(Conv2D(self.number_of_filters_base, self.filter_size,kernel_regularizer='l2', padding = self.padding, kernel_initializer = self.initializer))
        
            ## Double filter distribution - double number of filters in each Convolutional layers
            elif self.filter_distribution == "double":
                model.add(Conv2D(2**(i+1)*self.number_of_filters_base, self.filter_size, kernel_regularizer='l2',padding = self.padding, kernel_initializer = self.initializer))
        
            ## Halve the filter size in each successive convolutional layers
            elif self.filter_distribution == "half":
                model.add(Conv2D(int(self.number_of_filters_base/2**(i+1)), self.filter_size, kernel_regularizer='l2',padding = self.padding, kernel_initializer = self.initializer))
        
            model.add(Activation(self.activation))
        
            if self.batch_normalization: model.add(BatchNormalization())
        
            model.add(MaxPooling2D(pool_size=self.pool_size))
            if self.dropout_fraction != None:
                model.add(tf.keras.layers.Dropout(self.dropout_fraction))
        
        #Final densely connected layers
        model.add(Flatten())
        model.add(Dense(self.dense_neurons, activation = self.activation, kernel_regularizer='l2', kernel_initializer = self.initializer))
        model.add(Dense(self.num_classes, activation = 'softmax'))
        
        #model.compile(optimizer=self.optimizer,
        #      loss='categorical_crossentropy',
        #      metrics=['accuracy'])
        return model      
        
    def build_cnndropmodel2(self):
        keras.backend.clear_session()
        model = Sequential()
        
        #First CNN layer connecting to input layer
        model.add(Conv2D(self.number_of_filters_base, self.filter_size, kernel_regularizer='l2',padding = self.padding, input_shape = (self.IMG_HEIGHT, self.IMG_WIDTH, 3)))
        model.add(Activation(self.activation))
        
        #batch_normalisation
        if self.batch_normalization: model.add(BatchNormalization())
        #max pooling
        model.add(MaxPooling2D(pool_size=self.pool_size))  
        if self.dropout_fraction != None:
            model.add(tf.keras.layers.Dropout(self.dropout_fraction))
        for i in range(self.num_hidden_cnn_layers-1):
            #i+2th Convolutional Layer
        
            ## Standard filter distribution - same number of filters in all Convolutional layers
            if self.filter_distribution == "standard":
                model.add(Conv2D(self.number_of_filters_base, self.filter_size,kernel_regularizer='l2', padding = self.padding, kernel_initializer = self.initializer))
        
            ## Double filter distribution - double number of filters in each Convolutional layers
            elif self.filter_distribution == "double":
                model.add(Conv2D(2**(i+1)*self.number_of_filters_base, self.filter_size, kernel_regularizer='l2',padding = self.padding, kernel_initializer = self.initializer))
        
            ## Halve the filter size in each successive convolutional layers
            elif self.filter_distribution == "half":
                model.add(Conv2D(int(self.number_of_filters_base/2**(i+1)), self.filter_size, kernel_regularizer='l2',padding = self.padding, kernel_initializer = self.initializer))
        
            model.add(Activation(self.activation))
        
            if self.batch_normalization: model.add(BatchNormalization())
        
            model.add(MaxPooling2D(pool_size=self.pool_size))
            if self.dropout_fraction != None:
                model.add(tf.keras.layers.Dropout(self.dropout_fraction))
        
        #Final densely connected layers
        model.add(Flatten())
        model.add(Dense(self.dense_neurons, activation = self.activation, kernel_regularizer='l2', kernel_initializer = self.initializer))
        if self.dropout_fraction != None:
            model.add(tf.keras.layers.Dropout(self.dropout_fraction))
        model.add(Dense(self.num_classes, activation = 'softmax'))
        
        #model.compile(optimizer=self.optimizer,
        #      loss='categorical_crossentropy',
        #      metrics=['accuracy'])
        return model      
      
    def build_cnnmodelsimple(self):
        keras.backend.clear_session()
        model = Sequential()
        
        #First CNN layer connecting to input layer
        model.add(Conv2D(self.number_of_filters_base, self.filter_size, input_shape = (self.IMG_HEIGHT, self.IMG_WIDTH, 3)))
        model.add(Activation(self.activation))
        
        #batch_normalisation
        if self.batch_normalization: model.add(BatchNormalization())
        #max pooling
        model.add(MaxPooling2D(pool_size=self.pool_size))  
        for i in range(self.num_hidden_cnn_layers-1):
            #i+2th Convolutional Layer
        
            ## Standard filter distribution - same number of filters in all Convolutional layers
            if self.filter_distribution == "standard":
                model.add(Conv2D(self.number_of_filters_base, self.filter_size))
        
            ## Double filter distribution - double number of filters in each Convolutional layers
            elif self.filter_distribution == "double":
                model.add(Conv2D(2**(i+1)*self.number_of_filters_base, self.filter_size))
        
            ## Halve the filter size in each successive convolutional layers
            elif self.filter_distribution == "half":
                model.add(Conv2D(int(self.number_of_filters_base/2**(i+1)), self.filter_size))
        
            model.add(Activation(self.activation))
        
            if self.batch_normalization: model.add(BatchNormalization())
        
            model.add(MaxPooling2D(pool_size=self.pool_size))
        
        #Final densely connected layers
        model.add(Flatten())
        model.add(Dense(self.dense_neurons, activation = 'sigmoid'))
        if self.dropout_fraction != None:
            model.add(tf.keras.layers.Dropout(self.dropout_fraction))
        model.add(Dense(self.num_classes, activation = 'softmax'))
        
        #model.compile(optimizer=self.optimizer,
        #      loss='categorical_crossentropy',
        #      metrics=['accuracy'])
        return model 
        
    def build_cnnmodel(self):
        keras.backend.clear_session()
        model = Sequential()
        
        #First CNN layer connecting to input layer
        model.add(Conv2D(self.number_of_filters_base, self.filter_size, kernel_regularizer='l2',padding = self.padding, input_shape = (self.IMG_HEIGHT, self.IMG_WIDTH, 3)))
        model.add(Activation(self.activation))
        
        #batch_normalisation
        if self.batch_normalization: model.add(BatchNormalization())
        #max pooling
        model.add(MaxPooling2D(pool_size=self.pool_size))  
        for i in range(self.num_hidden_cnn_layers-1):
            #i+2th Convolutional Layer
        
            ## Standard filter distribution - same number of filters in all Convolutional layers
            if self.filter_distribution == "standard":
                model.add(Conv2D(self.number_of_filters_base, self.filter_size,kernel_regularizer='l2', padding = self.padding, kernel_initializer = self.initializer))
        
            ## Double filter distribution - double number of filters in each Convolutional layers
            elif self.filter_distribution == "double":
                model.add(Conv2D(2**(i+1)*self.number_of_filters_base, self.filter_size, kernel_regularizer='l2',padding = self.padding, kernel_initializer = self.initializer))
        
            ## Halve the filter size in each successive convolutional layers
            elif self.filter_distribution == "half":
                model.add(Conv2D(int(self.number_of_filters_base/2**(i+1)), self.filter_size, kernel_regularizer='l2',padding = self.padding, kernel_initializer = self.initializer))
        
            model.add(Activation(self.activation))
        
            if self.batch_normalization: model.add(BatchNormalization())
        
            model.add(MaxPooling2D(pool_size=self.pool_size))
        
        #Final densely connected layers
        model.add(Flatten())
        model.add(Dense(self.dense_neurons, activation = 'sigmoid', kernel_regularizer='l2'))
        if self.dropout_fraction != None:
            model.add(tf.keras.layers.Dropout(self.dropout_fraction))
        model.add(Dense(self.num_classes, activation = 'softmax'))
        
        #model.compile(optimizer=self.optimizer,
        #      loss='categorical_crossentropy',
        #      metrics=['accuracy'])
        return model      
        
        
    def load_pretrained_model(self):
        base_model = BASE_MODELS[self.base_model_name]
        base = base_model(weights='imagenet', include_top=False)
        x = base.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(self.dense_neurons, activation='relu')(x)
        guesses = Dense(self.num_classes, activation='softmax')(x)
        model = Model(inputs=base.input, outputs=guesses)

        # freeze all base layers
        for layer in base.layers:
            layer.trainable = False

        #model.compile(optimizer=self.optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        return model
 

In [6]:
# data preprocessing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import pathlib

#wandb logging
import wandb
from wandb.keras import WandbCallback


physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass




#data pre processing

data_augmentation = False

IMG_SIZE = (128,128)
BATCH_SIZE = 32


if data_augmentation == True:

#Faster Alternative
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255,
            validation_split = 0.1,
            shear_range=0.2,
            zoom_range=0.2,
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False
            )
else:
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split = 0.1)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/CS6910/Assignment2/inaturalist_12K/train',
    subset='training',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle = True,
     seed = 123)
    
validation_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/CS6910/Assignment2/inaturalist_12K/train',
        target_size=IMG_SIZE,
        subset = 'validation',
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle = True,
         seed = 123)


        
test_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/CS6910/Assignment2/inaturalist_12K/val',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle = True,
         seed = 123)

''' 
#sweep config
sweep_config = {
  "name": "Bayesian Sweep",
  "method": "bayes",
  "metric":{
  "name": "validationaccuracy",
  "goal": "maximize"
  },
  'early_terminate': {
        'type':'hyperband',
        'min_iter': [3],
        's': [2]
  },
  "parameters": {
        
        "activation":{
            "values": ["relu", "elu"]
        },
                    
        "batch_size": {
            "values": [32, 64]
        },
        "optimizer": {
            "values": ["sgd", "adam", "rmsprop"]
        },
        "batch_normalization": {
            "values": [True, False]
        },
        "number_of_filters_base": {
            "values": [32, 64]
        },
        "dense_neurons": {
            "values": [32, 64]
        },
        "dropout_fraction": {
            "values": [0.2,0.3]
        },        
    }
}

sweep_id = wandb.sweep(sweep_config,project='CS6910-DeepLearningFundamentals-Assignment1', entity='rahulsundar')

'''

#train function
def train():

        
    config_defaults = dict(
            num_hidden_cnn_layers = 5 ,
            activation = 'relu',
            batch_normalization = True,
            filter_distribution = "double" ,
            filter_size = (3,3),
            number_of_filters_base  = 32,
            initializer = 'he_uniform',
            dropout_fraction = None,
            pool_size = (2,2),
            padding = 'same',
            dense_neurons = 128,
            num_classes = 10,
            optimizer = 'adam',
            epochs = 5,
            batch_size = 32, 
            img_size = IMG_SIZE
        ) 
    wandb.init(project = 'CS6910-Assignment2-CNNs', config = config_defaults,entity='rahulsundar')
    CONFIG = wandb.config
        


    wandb.run.name = "OBJDET_" + str(CONFIG.num_hidden_cnn_layers) + "_dn_" + str(CONFIG.dense_neurons) + "_opt_" + CONFIG.optimizer + "_dro_" + str(CONFIG.dropout_fraction) + "_bs_"+str(CONFIG.batch_size) + "_fd_" + CONFIG.filter_distribution

    def myprint(s, path = '/content/drive/MyDrive/CS6910/Assignment2/TrainedModel/'+wandb.run.name):
        with open(path+"/mymodelsummary.txt",'w+') as f:
            print(s, file=f)
            
            
    objDetn = ObjectDetection(CONFIG.img_size, CONFIG )
    #model = objDetn.build_cnnmodel()
    model = objDetn.build_cnnmodelsimple()
    model.summary()



    model.compile(
    optimizer=CONFIG.optimizer,  # Optimizer
    # Loss function to minimize
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),#'categorical_crossentropy',
    # List of metrics to monitor
    metrics=['accuracy'],
    )
  
    history = model.fit(
                    train_generator,
                    steps_per_epoch = train_generator.samples // CONFIG.batch_size,
                    validation_data = validation_generator, 
                    validation_steps = validation_generator.samples // CONFIG.batch_size,
                    epochs = CONFIG.epochs, 
                    callbacks=[WandbCallback()]
                    )

    model.save('./TrainedModel/'+wandb.run.name)
    model.summary(print_fn=myprint)
    wandb.finish()
    return model, history
    


Found 9032 images belonging to 10 classes.
Found 1001 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


In [7]:
Model, History = train()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 126, 126, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 126, 126, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 61, 61, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 61, 61, 64)        2

KeyboardInterrupt: ignored